<h1>Research iteration 1</h1>

<i>Joël Boafo, Sjoerd Beetsma, Maarten de Jeu
Class V2A - Group 5</i>

<h2>Introduction</h2>

TODO(m-jeu): Place to write a quick introduction to the notebook structure, conventions, etc.

<h2>Business Understanding</h2>

Through the business, we have been asked to examine the following questions:

<ol>
<li>In hoeverre is de score van een Portugese Red te voorspellen op basis van de chemische kenmerken?</li>
<li>Zelf te bepalen: denk aan andere type wijnen of andere landen.</li>
<li>Zelf te bepalen: denk bijvoorbeeld aan het clusteren op basis van de chemische kenmerken waarmee het type druif of de regio bepaald kan worden. Of kun je logische clusters vinden van topwijnen, doordrinkwijnen en bocht?</li>
</ol>

<i>TODO(m-jeu): Either translate these questions to english, or change the language of the rest of the document to dutch. This also goes for the dutch list in 'Data Understanding'.</i>

Currently, the exact nature of 'the business' and their desires datascience-wise is unknown to us. This requires further investigation.

<h2> Data Understanding </h2>

The business tells us the most important variables in the dataset are:

<ol>
<li>Herkomst van de wijn en type druif.</li>
<li>Review van de wijn, inclusief naam van de sommelier en de score op een schaal van 1 tot 100.</li>
<li>De uitkomsten van chemische tests op 11 waarde (waaronder suikergehalte, pH, alcoholgehalte, et cetera).</li>

We import some libraries, and the dataset. Then we have an initial look at it:

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
dataset = pd.read_csv("datasets/redwine.csv", sep=";")
dataset.head(5)

Rows seems to correspond with individual wines on first glance, though this does need to be examined more thoroughly. Columns are different attributes for those individual wines.

<h3>Feature variables</h3>

Here we remove the columns: country, variety, description, title, taster_name and id. They are unnecessary for our model in determining the Target, which is points.
Country and variety are possible feature variables but are left out for now as the dataset contains only 1 unique country and variety.

In [ ]:
dataset_features_target = dataset.drop(['country', 'variety', 'description', 'title', 'taster_name', 'id'], axis=1)
dataset_features_target.head(0)

<h3> Scales of measurements </h3>

To construct an appropiate model it's necessary to have a understanding of all the scales of measurements for the target and feature variables.

In [ ]:
nom, disc, cont = 'Nominal', 'Discrete', 'Continous'
vars_scale = [nom, disc, disc, nom, nom, cont, cont, cont, cont, cont, cont, cont, cont, cont, cont, cont ]
measurement_scales = pd.DataFrame({'Variable':dataset_features_target.columns, 'Scale of measurement':vars_scale})
measurement_scales

<h3>Central tendancies and dispersion measures</h3>

From the central tendancies and dispersion measures we can see some useful statistics about the target and feature variables.

In [ ]:
dataset_features_target.describe().round(2)

To be noted the feature variables alcohol, density, citric acid aren't described even though they are of discrete scale, we will look at what's wrong during the data preparation phase.

<h3>Correlations</h3>

To help find positive, negative and neutral correlations matrix is constructed where dark red corresponds to a positive correlation and dark green a negative correlation.

In [ ]:
corr = dataset_features_target.corr()
plt.figure(figsize=(10,7.5))
cmap = sns.diverging_palette(200, 0, as_cmap=True) # color palette as cmap
mask = np.logical_not(np.tril(np.ones_like(corr))) # triangle mask
sns.heatmap(corr,annot=True, mask=mask, cmap = cmap, vmin=-1, vmax=1) # correlation heatmap

In the correlation matrix graph above you can see which attributes have a correlation to other attributes. Starting with our target variable 'quality', we can see quality has a few correlations with the strongest one being alcohol and a few weaker ones like volatile acidity, sulphates and citric acid. Because quality is our target variable it's indepented attribute in the correlation.

Besides there are some corelations among chemical properties:
Fixed acidity has strong correlation with pH, but it’s still an independent type. pH However is a dependent type; it depends on the former. Volatile acidity, residual sugar, sulphates, chlorides, and density are all independent data types. Total sulfur dioxide is dependent on free sulfur dioxide, but free sulfur dioxide is independent.

<h2>Data Preparation</h2>

The data needs some cleaning up. An overview of datatypes:

In [ ]:
dataset.dtypes

Jupyter doesn't recognize some of the Python str objects for what they are, and simply calls them the 'object' type. Let's convert them to the right type to allow for more method flexibility:

In [ ]:
dataset = dataset.convert_dtypes()
dataset.dtypes

There are some datatypes in the DataFrame that don't quite correspond to what you'd expect them to be, considering what they represent. Citric acid, Density and Alcohol are string objects, even though you'd expect them to be some kind of number-datatype. Let's copy everyone to a seperate column called 'raw_columnname' so that we can evaluate the original next to the converted in case we run into any problems turning these columns into numbers. Then we'll convert each one individually.

In [ ]:
problematic_column_names = ["citric acid", "density", "alcohol"]
copied_column_names = {f"raw_{name}":name for name in problematic_column_names}

for new, to_copy in copied_column_names.items():  # Is this allowed?
    dataset[new] = dataset[to_copy]

<h3>Data Preparation on column-by-column basis</h3>

<h4>Citric Acid</h4>

The citric acid column consists of string objects, through most entries are formatted like floats. Pandas can convert these for us, turning the ones that it can't understand into not-a-number entries.

In [ ]:
dataset["citric acid"]

In [ ]:
dataset["citric acid"] = pd.to_numeric(dataset["citric acid"], errors='coerce')

And let's define a simple convenience function that describes the min, max and amount of nan entries in a series object, to quickly gauge the validity of data contained within, and call it on the Citric Acid column.

In [ ]:
def simple_describe(s: pd.Series, name: str = "Series object") -> None:
    """Print out a simple description of a Pandas Series object that contains numeric values.
    That covers min, max and amount of nan/None entries.

    Args:
        s: Pandas series object with numeric values.
        name: optional name for the series to use in the printed description."""
    print(f"{name}:\nmin: {np.min(s)}\nmax: {np.max(s)}\n#(nan): {s.isnull().sum()}")

simple_describe(dataset["citric acid"], "Citric acid")

Auto converting the citric acid table to float leaves us with 203 not-a-number entries. Lets have a look at them.

In [ ]:
dataset.loc[dataset["citric acid"].isnull()][["id", "title", "citric acid", "raw_citric acid"]]

It appears these data points did not have a value for citric acid to begin with. According to <a href="https://wineserver.ucdavis.edu/industry-info/enology/methods-and-techniques/common-chemical-reagents/citric-acid">this website</a>, citric acid is something that's <i>added</i> to wine most of the time, but it doesn't seem essential. We'll assume that the wines with '-' for citric acid were given this value on purpose, and that this means that there's none in there. Let's replace those values with 0 in the raw table, and use that to create a final version of the citric acid table. Then we'll describe it again.

In [ ]:
dataset['raw_citric acid'] = dataset['raw_citric acid'].str.replace('-', '0')
dataset['citric acid'] = pd.to_numeric(dataset["raw_citric acid"], errors='coerce')
simple_describe(dataset['citric acid'])

Citric acid seems clean!

<h4>Density</h4>

Most density values are formatted like proper floats, so let's have pandas turn them into that, and then describe it. This time, the simple describe won't quite cut it.

In [ ]:
dataset["density"]

In [ ]:
dataset["density"] = pd.to_numeric(dataset["density"], errors='coerce')
dataset["density"].describe()

From the description, we see that almost all values (the mininum, and all quartiles) fall around the 0.99 range. Let's examine the spread of data further with the help of a frequency graph.

In [ ]:
plt.hist(dataset["density"], bins=50)  # TODO(m-jeu): Use OOP API and make clearer
plt.gca().set(title="Density frequency", xlabel="Density", ylabel="Frequency")
plt.show()

As we can see, most measurements fall around the '0.99' mark. Though some fall around the 10, and a couple fall around the 100 mark. We will assume that this is a notation error, where the dots weren't placed right, which can fairly easily be accounted for. If we assume that all measurements are supposed to be around '1', because it wouldn't be realistic for wines to have a 10x or 100x difference in density with others, we can correct for the mistake by recursively dividing the number by 10, until a more sensible measurement is reached.

In [ ]:
def divide_until_at(num: float,
                    target: float,
                    divide_by: float,
                    delta: float,
                    max_recursions: int = 0) -> float:
    """Recursively divide a number until it's either within a certain range of a target number,
    or the max number of recursions is reached.

    Args:
        num: input number to divide.
        target: target number to get within a certain range of.
        divide_by: number to divide input number by.
        delta: amount of distance the input number could be from the target number to be considered a success.
        max_recursion: amount of recursion after which the function should give up.

    Returns:
        num divided by divide_by a certain amount of times, so that it's within delta of target.
        amount of times num is divided can also be 0.
        None if max_recursions gets violated."""
    if max_recursions < 0:
        return None
    if abs(num - target) < delta:
        return num
    return divide_until_at((num / divide_by), target, divide_by, delta, (max_recursions - 1))

dataset["density"] = dataset["density"].apply(lambda x: divide_until_at(x, 1, 10, 0.1, 2))
dataset["density"].describe()

In [ ]:
simple_describe(dataset["density"], "Density")

The weird numbers have been taken care of. A single not-a-number entry remains. Let's have a look:

In [ ]:
dataset.loc[dataset["density"].isnull()][['id', 'title', 'density', 'raw_density']]

A single wine has a density of '.', which we can't really use for anything. To keep the data pure, let's drop it.

In [ ]:
dataset = dataset[dataset["density"].notna()]
simple_describe(dataset["density"], "Density")

Density is clean!

<h4>Alcohol</h4>

Most alcohol percentages are formatted like proper floats, and can be converted automatically. This allows us to analyse the actual numbers as well:

In [ ]:
dataset["alcohol"] = pd.to_numeric(dataset["alcohol"], errors='coerce')  # No OOP interface.
simple_describe(dataset["alcohol"], "Alcohol")

The alcohol-percentages we have been able to extract from the String objects seem to fall within the boundaries of what could reasonably be considered 'wine'. We are still left with 10 not-a-numbers entries where pandas wasn't able to convert the strings to numbers. Let's have a look at them.

In [ ]:
dataset.loc[dataset["alcohol"].isnull()][['id', 'title', 'designation', 'alcohol']]  # TODO(m-jeu): This can probably be made prettier

Considering only 10 wines are missing data, we can probably look these up by hand. The percentages are based on these sources:

<a href="https://www.internationalwinechallenge.com/canopy/beverage_details?wid=44074">1344. </a><a href="https://www.winemag.com/buying-guide/companhia-das-quintas-2009-quinta-da-fronteira-seleccao-do-enologo-red-douro/">1345. </a><a href="https://www.winemag.com/buying-guide/quinta-das-bandeiras-2010-passagem-reserva-red-douro/">1620. </a><a href="https://www.nataliemaclean.com/wine-reviews/quinta-da-rede-reserva-2015/317025">1670. </a><a href="https://www.winemag.com/buying-guide/terra-silvestre-2014-grande-reserva-red-tejo-portuguese-red/">1764. </a><a href="https://www.vivino.com/NL/en/adega-cooperativa-de-borba-alentejo-reserva-tinto/w/1235048?year=2013">1765. </a><a href="https://www.winemag.com/buying-guide/adega-cooperativa-cartaxo-2012-bridao-classico-red-tejo/">1766. </a><a href="https://www.winemag.com/buying-guide/quinta-da-lagoalva-de-cima-2015-lagoalva-barrel-selection-red-tejo/">1794. </a><a href="https://www.winemag.com/buying-guide/quinta-do-casal-branco-2014-lobo-e-falcao-reserva-red-tejo/">1795. </a><a href="https://www.wine-searcher.com/find/ramos+pinto+duas+quinta+rsrv+douro+portugal/2006/netherlands#t2">2028. </a>

In [ ]:
# Assignments use numeric row index, and not 'id' column. Might break when working on the id column.
dataset.loc[1344, "alcohol"] = 14.0
dataset.loc[1345, "alcohol"] = 14.5
dataset.loc[1620, "alcohol"] = 14.0
dataset.loc[1670, "alcohol"] = 13.5
dataset.loc[1764, "alcohol"] = 13.5
dataset.loc[1765, "alcohol"] = 14.0
dataset.loc[1766, "alcohol"] = 14.0
dataset.loc[1794, "alcohol"] = 14.0
dataset.loc[1795, "alcohol"] = 14.0
dataset.loc[2028, "alcohol"] = 14.0

Lets' have a look how many nan entries are left in the alcohol column:

In [ ]:
dataset["alcohol"].isnull().sum()

<h3>Dropping unneeded columns</h3>

We drop columns description, title, taster_name and id, because they are not useful for later models, and hold no sway over our target variable, points. We also drop raw_citric acid, raw_density and raw_alcohol because they are no longer needed in data preparation.

Leaving us with a dataset containing clean target and feature variables.


In [ ]:
dataset = dataset.drop(['country', 'variety', 'description', 'title', 'taster_name', 'id', 'raw_citric acid', 'raw_density', 'raw_alcohol'], axis=1)

In [ ]:
dataset.head(5)

TODO(sbeetsma): 'Alentejano' probably doesn't exist as province, and they mean 'Alentejo'. Should probably check this and add source before implementing.